In [3]:
import mujoco as mj
from mujoco.glfw import glfw
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio

xml_path = 'controlPos.xml'
simend = 10

# For callback functions
button_left = False
button_middle = False
button_right = False
lastx = 0
lasty = 0

posiciones = []
velocidades = []
tiempos = []
control = []

def controller(model, data):
    """
    This function implements a PD controller

    Since there are no gravity compensation,
    it will not be very accurate at tracking
    the set point. It will be accurate if
    gravity is turned off.
    """
    ctrl_phase = 2 * np.pi  # Control phase
    ctrl_freq = 0.5     # Control frequency
    
    #print(np.random.rand(model.nu))
    
    if actuator_type == "torque":
        model.actuator_gainprm[0, 0] = 1
        data.ctrl[0] = np.sin(ctrl_phase + 2 * np.pi * data.time * ctrl_freq)
        #posiciones.append(data.qpos)
        #velocidades.append(data.qvel)
        #tiempos.append(data.time)

    elif actuator_type == "servo":
        kp = 10.0
        model.actuator_gainprm[1, 0] = kp # kp 0 0
        model.actuator_biasprm[1, 1] = -kp # 0 -kp -kv
        data.ctrl[1] = np.pi/2

        #kv = 1.0
        #model.actuator_gainprm[2, 0] = kv
        #model.actuator_biasprm[2, 2] = -kv
        #data.ctrl[2] = 0.2 # m/s, setpoint of the velocity servo, initial velocity?

def keyboard(window, key, scancode, act, mods):
    if act == glfw.PRESS and key == glfw.KEY_BACKSPACE:
        mj.mj_resetData(model, data)
        mj.mj_forward(model, data)

        
def mouse_button(window, button, act, mods):
        # update button state
        button_left = (glfw.get_mouse_button(
            window, glfw.MOUSE_BUTTON_LEFT) == glfw.PRESS)
        button_middle = (glfw.get_mouse_button(
            window, glfw.MOUSE_BUTTON_MIDDLE) == glfw.PRESS)
        button_right = (glfw.get_mouse_button(
            window, glfw.MOUSE_BUTTON_RIGHT) == glfw.PRESS)

        # update mouse position
        glfw.get_cursor_pos(window)

def mouse_move(window, xpos, ypos):
    # compute mouse displacement, save
    global lastx
    global lasty
    dx = xpos - lastx
    dy = ypos - lasty
    lastx = xpos
    lasty = ypos

    # no buttons down: nothing to do
    if (not button_left) and (not button_middle) and (not button_right):
        return

    # get current window size
    width, height = glfw.get_window_size(window)

    # get shift key state
    PRESS_LEFT_SHIFT = glfw.get_key(
        window, glfw.KEY_LEFT_SHIFT) == glfw.PRESS
    PRESS_RIGHT_SHIFT = glfw.get_key(
        window, glfw.KEY_RIGHT_SHIFT) == glfw.PRESS
    mod_shift = (PRESS_LEFT_SHIFT or PRESS_RIGHT_SHIFT)

    # determine action based on mouse button
    if button_right:
        if mod_shift:
            action = mj.mjtMouse.mjMOUSE_MOVE_H
        else:
            action = mj.mjtMouse.mjMOUSE_MOVE_V
    elif button_left:
        if mod_shift:
            action = mj.mjtMouse.mjMOUSE_ROTATE_H
        else:
            action = mj.mjtMouse.mjMOUSE_ROTATE_V
    else:
        action = mj.mjtMouse.mjMOUSE_ZOOM

    mj.mjv_moveCamera(model, action, dx/height,
                      dy/height, scene, cam)

def scroll(window, xoffset, yoffset):
    action = mj.mjtMouse.mjMOUSE_ZOOM
    mj.mjv_moveCamera(model, action, 0.0, -0.05 *
                      yoffset, scene, cam)



In [2]:

#get the full path
#dirname = os.path.dirname(__file__)
dirname = os.getcwd() # Use getcwd() to get the current working directory
#abspath = os.path.join(dirname + "/" + xml_path)
abspath = os.path.join(dirname, xml_path)
xml_path = abspath

# MuJoCo data structures
model = mj.MjModel.from_xml_path(xml_path)  # MuJoCo model
data = mj.MjData(model)                # MuJoCo data
cam = mj.MjvCamera()                        # Abstract camera
opt = mj.MjvOption()                        # visualization options

# Init GLFW, create window, make OpenGL context current, request v-sync
glfw.init()
window = glfw.create_window(1200, 900, "Demo", None, None)
glfw.make_context_current(window)
glfw.swap_interval(1)

# initialize visualization data structures
mj.mjv_defaultCamera(cam)
mj.mjv_defaultOption(opt)
scene = mj.MjvScene(model, maxgeom=10000)
context = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)

# install GLFW mouse and keyboard callbacks
glfw.set_key_callback(window, keyboard)
glfw.set_cursor_pos_callback(window, mouse_move)
glfw.set_mouse_button_callback(window, mouse_button)
glfw.set_scroll_callback(window, scroll)

#set initial conditions
data.qpos[0] = np.pi/2 # to only one degree of freedom '0'

# Set camera configuration
cam.azimuth = 90.0
cam.distance = 5.0
cam.elevation = -5
cam.lookat = np.array([0.012768, -0.000000, 1.254336])

#set the controller
actuator_type = "torque"
mj.set_mjcb_control(controller)

###########################
tiempouno = []

#############
while not glfw.window_should_close(window):
    simstart = data.time
    tiempouno.append(data.time)

    while (data.time - simstart < 0.5/60.0):
        mj.mj_step(model, data)
        #################
        posiciones.append(data.qpos[0])
        velocidades.append(data.qvel[0])
        control.append(data.ctrl[0])
        tiempos.append(data.time)
        ####################
        
    if (data.time>=simend):
        break;

    # get framebuffer viewport
    viewport_width, viewport_height = glfw.get_framebuffer_size(
        window)
    viewport = mj.MjrRect(0, 0, viewport_width, viewport_height)

    # Update scene and render
    mj.mjv_updateScene(model, data, opt, None, cam,
                       mj.mjtCatBit.mjCAT_ALL.value, scene)
    mj.mjr_render(viewport, scene, context)

    # swap OpenGL buffers (blocking call due to v-sync)
    glfw.swap_buffers(window)

    # process pending GUI events, call GLFW callbacks
    glfw.poll_events()

glfw.terminate()

In [ ]:
# Crear el gráfico
plt.plot(tiempos, posiciones, color='b', label='Position')

# Agregar etiquetas y título
plt.xlabel('Time (s)')
plt.ylabel('Position (rad)')
#plt.title('Gráfico de Tiempo vs Posición')
plt.legend()  # Mostrar la leyenda
plt.grid(True)  # Agregar una cuadrícula
plt.savefig('OL_SineControlSignal_Pos.png', dpi=300, bbox_inches='tight', format='png')

# Mostrar el gráfico
plt.show()


In [ ]:
# Crear el gráfico
plt.plot(tiempos, velocidades, color='b', label='Velocity')

# Agregar etiquetas y título
plt.xlabel('Time (s)')
plt.ylabel('Velocity (rad/s)')
#plt.title('Gráfico de Tiempo vs Velocidad')
plt.legend()  # Mostrar la leyenda
plt.grid(True)  # Agregar una cuadrícula
plt.savefig('OL_SineControlSignal_Vel.png', dpi=300, bbox_inches='tight', format='png')


# Mostrar el gráfico
plt.show()

In [ ]:
# Crear el gráfico
plt.plot(tiempos, control, color='b', label='Control')

# Agregar etiquetas y título
plt.xlabel('Time (s)')
plt.ylabel('Control signal')
#plt.title('Gráfico de Tiempo vs Control')
#plt.legend()  # Mostrar la leyenda
plt.grid(True)  # Agregar una cuadrícula
plt.savefig('OL_SineControlSignal_ContSign.png', dpi=300, bbox_inches='tight', format='png')


# Mostrar el gráfico
plt.show()

In [ ]:
# SEñal de control y posición
plt.plot(tiempos, posiciones, label='Position', color='b')  # Graficar señal 1
plt.plot(tiempos, control, label='Control', color='r')  # Graficar señal 2

# Agregar etiquetas y título

plt.xlabel('Time (s)')
plt.ylabel('Angular position (rad)')
#plt.title('Gráfico de dos señales con respecto al tiempo')
plt.legend()  # Mostrar la leyenda para distinguir las señales
plt.grid(True)  # Agregar una cuadrícula para facilitar la lectura
plt.savefig('OL_SineControlSignal_PosVsCtrl.png', dpi=300, bbox_inches='tight', format='png')


# Mostrar el gráfico
plt.show()

# Humanoid try, did not work

In [ ]:
# Get MuJoCo's standard humanoid model.
print('Getting MuJoCo humanoid XML description from GitHub:')
#!git clone https://github.com/google-deepmind/mujoco
with open('humanoid.xml', 'r') as f:
  xml = f.read()

# Load the model, make two MjData's.
model = mj.MjModel.from_xml_path(xml)
data = mj.MjData(model)
data2 = mj.MjData(model)

# Episode parameters.
duration = 3       # (seconds)
framerate = 60     # (Hz)
data.qpos[0:2] = [-.5, -.5]  # Initial x-y position (m)
data.qvel[2] = 4   # Initial vertical velocity (m/s)
ctrl_phase = 2 * np.pi * np.random.rand(model.nu)  # Control phase
ctrl_freq = 1     # Control frequency

# Visual options for the "ghost" model.
vopt2 = mj.MjvOption()
vopt2.flags[mj.mjtVisFlag.mjVIS_TRANSPARENT] = True  # Transparent.
pert = mj.MjvPerturb()  # Empty MjvPerturb object
# We only want dynamic objects (the humanoid). Static objects (the floor)
# should not be re-drawn. The mjtCatBit flag lets us do that, though we could
# equivalently use mjtVisFlag.mjVIS_STATIC
catmask = mj.mjtCatBit.mjCAT_DYNAMIC

# Simulate and render.
frames = []
with mj.Renderer(model, 480, 640) as renderer:
  while data.time < duration:
    # Sinusoidal control signal.
    data.ctrl = np.sin(ctrl_phase + 2 * np.pi * data.time * ctrl_freq)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      # This draws the regular humanoid from `data`.
      renderer.update_scene(data)

      # Copy qpos to data2, move the humanoid sideways, call mj_forward.
      data2.qpos = data.qpos
      data2.qpos[0] += 1.5
      data2.qpos[1] += 1
      mj.mj_forward(model, data2)

      # Call mjv_addGeoms to add the ghost humanoid to the scene.
      mj.mjv_addGeoms(model, data2, vopt2, pert, catmask, renderer.scene)

      # Render and add the frame.
      pixels = renderer.render()
      frames.append(pixels)

# Render video at half real-time.
media.show_video(frames, fps=framerate/2)